# Summary
- Target categories:
    - Good (suff. rating >= 80%)
    - Fair (suff. rating < 80% and >= 50%)
    - Poor (suff. rating < 50%)

# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## sklearn

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
# models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
# tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# model save
from sklearn.externals import joblib

## other

In [3]:
# upsampling with SMOTE
from imblearn import over_sampling
from imblearn.pipeline import Pipeline as imbPipeline

# Plot Settings

In [4]:
# format style
plt.style.use('fivethirtyeight')
# lineweight
plt.rc('lines', linewidth=3)
# figure size
plt.rc('figure', figsize=(12, 7))
# title fontsize
plt.rc('axes', titlesize=33) 
# axes label fontsize
plt.rc('axes', labelsize=28)
# axes values fontsize
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
# legend fontsize
plt.rc('legend', fontsize=18)

# Custom Functions

In [5]:
def rat_conv(rating):
    '''
    assign sufficiency rating to category
    '''
    if rating < 50:
        return('poor')
    else:
        return ('not poor')

# Space Log
Return range of numbers between two values in a log scale

In [6]:
def space_log(start, stop, number):
    return(np.exp(np.linspace(np.log(start), np.log(stop), number)))

## Feature Weight Sorting

In [46]:
def feat_sort(values, labels, num_ret='all', ret_abs=False, ret_pct=False, ret_0=False):
    '''
    Return sorted (descending) feature weights
    
    Parameters
    ----------
    values : feature weight values from analysis
    labels : names of each feature
    num_ret : number of top features to return
    ret_abs : return absolute value of weight values (will track original sign)
    ret_pct : return each weight as percentage of total (based on absolute value)
    ret_0 : return weights with value of zero
    '''
    
    # create feature weight dataframe
    df = pd.DataFrame(values, index=labels, columns=['feat_wgt']).copy()
        
    # ret_abs check
    if ret_abs == True:
        # create column identifying positive weights
        df['positive'] = df['feat_wgt'] > 0
        # transform weights to absolute value
        df['feat_wgt'] = df['feat_wgt'].apply(abs)
        
    # sort weights (largest to smallest)
    df.sort_values(by='feat_wgt', ascending=False, inplace=True)
    
    # ret_pct check
    if ret_pct == True:
        if ret_abs == False:
            df['feat_wgt'] = df['feat_wgt'].apply(abs)
            df.sort_values(by='feat_wgt', ascending=False, inplace=True)
        # transform weights to percentages
        df['feat_wgt'] = round(df['feat_wgt'] / df['feat_wgt'].sum() * 100, 0).astype(int)
        
    # ret_0 check
    if ret_0 == False:
        # drop weights == 0
        df = df[df['feat_wgt'] != 0]
        
    # return dataframe
    if num_ret == 'all':
        return(df)
    else:
        return(df.iloc[:num_ret, :])

# Data

In [8]:
df = pd.read_csv('data/bridges.csv', index_col=0)
df.head()

,COUNTY_CODE_003,FEATURES_DESC_006A,FACILITY_CARRIED_007,LOCATION_009,MIN_VERT_CLR_010,DETOUR_KILOS_019,AGE,TRAFFIC_LANES_ON_028A,TRAFFIC_LANES_UND_028B,ADT_029,...,APPR_KIND_044A,APPR_TYPE_044B,DECK_STRUCTURE_TYPE_107,SURFACE_TYPE_108A,MEMBRANE_TYPE_108B,DECK_PROTECTION_108C,DECK_COND_058,SUPERSTRUCTURE_COND_059,SUBSTRUCTURE_COND_060,SUFFICIENCY_RATING_tar_yr
51-1VA0158,99.0,'Gambo Creek ','Tisdale Rd ','1km N of Dahlgren Rd ',99.99,5.0,67.0,2.0,0,1650.0,...,0.0,0.0,1,0,0,0,5,4,4,13.0
51-1VA0575,740.0,'Dale St. & N&P RR ','Williams Avenue ','At Gate 36 and Elm Ave. ',99.99,1.0,65.0,2.0,2,5000.0,...,5.0,1.0,1,6,0,0,5,5,5,7.0
51-1VA0591,810.0,'Drainage Canal ','Golf Cart Path ','Near 9th Hole ',99.99,1.0,42.0,2.0,0,50.0,...,0.0,0.0,1,0,0,0,5,5,6,48.0
51-1VA2106,810.0,'Lake Whitehurst Outlet ','Guam Road ','1 KM NW of Ferry Rd ',99.99,5.0,53.0,2.0,0,500.0,...,0.0,0.0,1,0,0,0,5,6,5,79.5
51-1VA2107,810.0,'Eastern Shore RR ','Amphibious Drive ','0.2 KM W of Abbott Rd ',99.99,2.0,52.0,2.0,2,4000.0,...,3.0,2.0,1,0,0,0,5,5,4,63.4


In [9]:
# number of observations
len(df)

7803

# Pre-processing

In [10]:
# bridge identification
df_id = df.iloc[:, 0:4]

In [11]:
# numerical features
num_cols = []
X_num = df.loc[:, 'MIN_VERT_CLR_010':'SUFFICIENCY_RATING_feat_yr']

## Categorical Data

In [12]:
# categorical features
cat_cols = []
X_cat = df.loc[:, 'TOLL_020':'SUBSTRUCTURE_COND_060']

In [13]:
X_cat_enc = pd.get_dummies(X_cat)
X_cat_enc.head()

,TOLL_020,MAINTENANCE_021,FUNCTIONAL_CLASS_026,DESIGN_LOAD_031,MEDIAN_CODE_033,STRUCTURE_FLARED_035,HISTORY_037,SERVICE_ON_042A,SERVICE_UND_042B,STRUCTURE_KIND_043A,...,SUPERSTRUCTURE_COND_059_9,SUPERSTRUCTURE_COND_059_N,SUBSTRUCTURE_COND_060_3,SUBSTRUCTURE_COND_060_4,SUBSTRUCTURE_COND_060_5,SUBSTRUCTURE_COND_060_6,SUBSTRUCTURE_COND_060_7,SUBSTRUCTURE_COND_060_8,SUBSTRUCTURE_COND_060_9,SUBSTRUCTURE_COND_060_N
51-1VA0158,3.0,73.0,9.0,8.0,0.0,0.0,5.0,1,5,2,...,0,0,0,1,0,0,0,0,0,0
51-1VA0575,3.0,73.0,19.0,0.0,0.0,0.0,4.0,5,4,1,...,0,0,0,0,1,0,0,0,0,0
51-1VA0591,3.0,73.0,19.0,0.0,0.0,0.0,5.0,1,0,3,...,0,0,0,0,0,1,0,0,0,0
51-1VA2106,3.0,73.0,19.0,5.0,0.0,0.0,5.0,1,5,3,...,0,0,0,0,1,0,0,0,0,0
51-1VA2107,3.0,73.0,19.0,6.0,0.0,0.0,5.0,5,4,4,...,0,0,0,1,0,0,0,0,0,0


## Combine Categorical and Numeric Data

In [14]:
X = pd.concat([X_num, X_cat_enc], axis=1)
X.head()

,MIN_VERT_CLR_010,DETOUR_KILOS_019,AGE,TRAFFIC_LANES_ON_028A,TRAFFIC_LANES_UND_028B,ADT_029,APPR_WIDTH_MT_032,DEGREES_SKEW_034,NAV_VERT_CLR_MT_039,NAV_HORR_CLR_MT_040,...,SUPERSTRUCTURE_COND_059_9,SUPERSTRUCTURE_COND_059_N,SUBSTRUCTURE_COND_060_3,SUBSTRUCTURE_COND_060_4,SUBSTRUCTURE_COND_060_5,SUBSTRUCTURE_COND_060_6,SUBSTRUCTURE_COND_060_7,SUBSTRUCTURE_COND_060_8,SUBSTRUCTURE_COND_060_9,SUBSTRUCTURE_COND_060_N
51-1VA0158,99.99,5.0,67.0,2.0,0,1650.0,7.3,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
51-1VA0575,99.99,1.0,65.0,2.0,2,5000.0,8.1,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
51-1VA0591,99.99,1.0,42.0,2.0,0,50.0,6.1,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
51-1VA2106,99.99,5.0,53.0,2.0,0,500.0,9.1,9.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
51-1VA2107,99.99,2.0,52.0,2.0,2,4000.0,7.9,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


## Target Data

In [15]:
# set target column values to categories
y = df.iloc[:, -1].apply(rat_conv)

In [16]:
# number in each category
y.value_counts()

not poor    6938
poor         865
Name: SUFFICIENCY_RATING_tar_yr, dtype: int64

## Train/Test Split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Initial Modeling

## Logistic Regression

In [18]:
# model pipeline includes scaling
log_pipe = Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('model', LogisticRegression(class_weight='balanced'))
])

In [19]:
log_pipe.fit(X_train, y_train)
log_pred = log_pipe.predict(X_test)

In [20]:
print(metrics.classification_report(y_test, log_pred))

             precision    recall  f1-score   support

   not poor       0.98      0.88      0.93      1375
       poor       0.49      0.89      0.63       186

avg / total       0.92      0.88      0.89      1561



In [21]:
metrics.confusion_matrix(y_test, log_pred)

array([[1204,  171],
       [  21,  165]])

In [22]:
log_pipe.classes_

array(['not poor', 'poor'], dtype=object)

In [23]:
log_imp = feat_sort(log_pipe.named_steps['model'].coef_[0], X_train.columns, ret_abs=True, ret_pct=True)
log_imp

,feat_wgt,positive
SUFFICIENCY_RATING_feat_yr,8.6,False
STRUCTURE_LEN_MT_049,4.9,False
ROADWAY_WIDTH_MT_051,4.7,False
APPR_RAIL_END_036D_N,4.0,False
TRAFFIC_LANES_ON_028A,3.5,True
NAV_HORR_CLR_MT_040,3.3,True
TOLL_020,3.0,True
SUPERSTRUCTURE_COND_059_9,2.9,False
SUPERSTRUCTURE_COND_059_8,2.4,False
TRANSITIONS_036B_N,2.3,True


### Refine

In [24]:
# try range of values for C and penalty hyperparameters
hyperparams = {'model__penalty':['l1', 'l2'], 'model__C': space_log(0.001, 100, 15)}

In [25]:
# tune model pipeline for recall
log_tune = GridSearchCV(log_pipe, hyperparams, cv=5)

In [26]:
log_tune.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'model__penalty': ['l1', 'l2'], 'model__C': array([1.00000e-03, 2.27585e-03, 5.17947e-03, 1.17877e-02, 2.68270e-02,
       6.10540e-02, 1.38950e-01, 3.16228e-01, 7.19686e-01, 1.63789e+00,
       3.72759e+00, 8.48343e+00, 1.93070e+01, 4.39397e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
log_tune_pred = log_tune.predict(X_test)

In [28]:
# assign best hyperparameter values from tuning
best_penalty = log_tune.best_params_['model__penalty']
best_C = log_tune.best_params_['model__C']

In [29]:
best_penalty

'l1'

In [30]:
best_C

0.02682695795279727

In [31]:
print(metrics.classification_report(y_test, log_tune_pred))

             precision    recall  f1-score   support

   not poor       0.98      0.88      0.93      1375
       poor       0.50      0.89      0.64       186

avg / total       0.93      0.88      0.89      1561



In [32]:
metrics.confusion_matrix(y_test, log_pred)

array([[1204,  171],
       [  21,  165]])

In [33]:
# model pipeline with optimal hyperparameters
log_pipe = Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('model', LogisticRegression(penalty=best_penalty, C=best_C, class_weight='balanced'))
])

In [34]:
log_pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('model', LogisticRegression(C=0.02682695795279727, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [47]:
tune_feats = feat_sort(log_pipe.named_steps['model'].coef_[0], X_train.columns, ret_abs=True, ret_pct=True)
tune_feats

,feat_wgt,positive
SUFFICIENCY_RATING_feat_yr,35,False
DESIGN_LOAD_031,6,False
OPEN_CLOSED_POSTED_041_P,6,True
DECK_STRUCTURE_TYPE_107_8,6,True
SUPERSTRUCTURE_COND_059_5,5,True
SUPERSTRUCTURE_COND_059_6,5,True
SUPERSTRUCTURE_COND_059_8,5,False
RECON_AGE,3,True
FUNCTIONAL_CLASS_026,3,True
RIGHT_CURB_MT_050B,2,True


In [48]:
len(tune_feats)

28

In [49]:
feat_sort(log_pipe.named_steps['model'].coef_[0], X_train.columns)

,feat_wgt
OPEN_CLOSED_POSTED_041_P,0.266713
DECK_STRUCTURE_TYPE_107_8,0.262410
SUPERSTRUCTURE_COND_059_5,0.225325
SUPERSTRUCTURE_COND_059_6,0.210265
RECON_AGE,0.140815
FUNCTIONAL_CLASS_026,0.140724
RIGHT_CURB_MT_050B,0.108026
APPR_KIND_044A,0.079388
SURFACE_TYPE_108A_6,0.072047
TRANSITIONS_036B_N,0.070708


## Random Forest

In [41]:
# model pipeline includes scaling
rf_pipe = imbPipeline([
    ('oversample', over_sampling.SMOTE(random_state=0)),
    ('model', RandomForestClassifier(random_state=0))
])

In [42]:
rf_pipe.fit(X_train, y_train)
rf_pred = rf_pipe.predict(X_test)

In [43]:
print(metrics.classification_report(y_test, rf_pred))

             precision    recall  f1-score   support

   not poor       0.94      0.96      0.95      1375
       poor       0.68      0.57      0.62       186

avg / total       0.91      0.92      0.91      1561



In [44]:
metrics.confusion_matrix(y_test, rf_pred)

array([[1325,   50],
       [  80,  106]])

In [31]:
rf_imp = feat_sort(rf_pipe.named_steps['model'].feature_importances_, X_train.columns, 20)
rf_imp

,feat_wgt,positive
SUFFICIENCY_RATING_feat_yr,0.161544,True
OPEN_CLOSED_POSTED_041_P,0.053668,True
RAILINGS_036A_0,0.049126,True
AGE,0.046734,True
RECON_AGE,0.037778,True
DECK_WIDTH_MT_052,0.037561,True
ROADWAY_WIDTH_MT_051,0.033556,True
OPEN_CLOSED_POSTED_041_A,0.031948,True
ADT_029,0.031877,True
RAILINGS_036A_1,0.029135,True


## Refine

In [76]:
# try range of values for hyperparameters

# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features to consider at every split
max_features = np.arange(1, X.shape[1]+1)
# maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# method of selecting samples for training each tree
bootstrap = [True, False]

In [77]:
hyperparams = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [81]:
# tune model pipeline for recall
rf_tune = RandomizedSearchCV(RandomForestClassifier(random_state=0), hyperparams, cv=5, random_state=0)

In [82]:
rf_tune.fit(X_train, y_train)
rf_pred = rf_tune.predict(X_test)

In [83]:
print(metrics.classification_report(y_test, rf_pred))

             precision    recall  f1-score   support

       fair       0.70      0.72      0.71       726
       good       0.84      0.84      0.84       993
       poor       0.68      0.57      0.62       232

avg / total       0.77      0.77      0.77      1951



In [84]:
metrics.confusion_matrix(y_test, rf_pred)

array([[526, 154,  46],
       [140, 837,  16],
       [ 90,   9, 133]])

In [85]:
rf_tune.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 91,
 'max_depth': 50,
 'bootstrap': True}

In [87]:
# model pipeline includes scaling
rf_pipe = imbPipeline([
    ('oversample', over_sampling.SMOTE(random_state=0)),
    ('model', RandomForestClassifier(
        random_state=0,
        n_estimators=rf_tune.best_params_['n_estimators'],
        min_samples_split=rf_tune.best_params_['min_samples_split'],
        min_samples_leaf=rf_tune.best_params_['min_samples_leaf'],
        max_features=rf_tune.best_params_['max_features'],
        max_depth=rf_tune.best_params_['max_depth']
    ))
])

In [88]:
rf_pipe.fit(X_train, y_train)
rf_pred = rf_pipe.predict(X_test)

In [89]:
print(metrics.classification_report(y_test, rf_pred))

             precision    recall  f1-score   support

       fair       0.68      0.72      0.70       726
       good       0.84      0.81      0.83       993
       poor       0.65      0.62      0.64       232

avg / total       0.76      0.76      0.76      1951



In [90]:
metrics.confusion_matrix(y_test, rf_pred)

array([[526, 142,  58],
       [169, 805,  19],
       [ 80,   7, 145]])

In [95]:
rf_imp = feat_sort(rf_pipe.named_steps['model'].feature_importances_, X_train.columns)
rf_imp

,feat_wgt,positive
OPEN_CLOSED_POSTED_041_A,0.258519,True
DESIGN_LOAD_031,0.103882,True
ADT_029,0.075570,True
AGE,0.050973,True
ROADWAY_WIDTH_MT_051,0.044290,True
DETOUR_KILOS_019,0.039338,True
APPR_WIDTH_MT_032,0.025697,True
STRUCTURE_LEN_MT_049,0.025580,True
DECK_WIDTH_MT_052,0.024942,True
APPR_SPANS_046,0.023370,True


## SVM

In [61]:
# model pipeline includes scaling
svm_pipe = Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('model', svm.SVC(class_weight='balanced'))
])

In [62]:
svm_pipe.fit(X_train, y_train)
svm_pred = svm_pipe.predict(X_test)

In [63]:
print(metrics.classification_report(y_test, svm_pred))

             precision    recall  f1-score   support

   not poor       0.97      0.90      0.93      1375
       poor       0.52      0.83      0.64       186

avg / total       0.92      0.89      0.90      1561



In [64]:
metrics.confusion_matrix(y_test, svm_pred)

array([[1234,  141],
       [  32,  154]])